In [1]:
from util import *
from trueskill import Rating, rate, BETA, global_env, setup, quality
import itertools
import math
import pandas as pd
# 根据 Trueskill 的 rating 模型，计算胜率
def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [2]:
data = load_json('data/wc_sofifa.json')
df = pd.DataFrame(data)[['name', 'overall', 'positions', 'club']]

In [7]:
elos = load_json('data/elo.json')
df['elo'] = df['club'].apply(lambda x: elos[x])
df[df['club'] == 'Peru']

,name,overall,positions,club,elo
325,J. Farfán,80,"[ST, RM]",Peru,1915
350,A. Carrillo,79,[RM],Peru,1915
458,C. Cueva,78,[CAM],Peru,1915
494,R. Tapia,77,[CDM],Peru,1915
598,C. Pizarro,76,[ST],Peru,1915
647,P. Gallese,76,[GK],Peru,1915
675,R. Ruidíaz,76,[ST],Peru,1915
676,L. Advíncula,76,[RB],Peru,1915
685,E. Flores,75,[LM],Peru,1915
743,A. Rodríguez,75,[CB],Peru,1915


In [8]:
matchups = [
    {
    'Denmark':[83,80,82,71,73,77,78,89,76,78,74],
    'Peru':[76,75,74,74,76,78,77,75,80,79,75]   
    },
]
# 0.22 来自所有国家队比赛数据
setup(draw_probability=0.22, sigma=2000/3, beta=2000/3/2)
def calculate_with_matchup(matchup):
    teams = []
    nations = []
    for nation, indexes in matchup.items():
        ratings = []
        nations.append(nation)
#         print(nation)
        for i in indexes:
            player = df.iloc[i]
            ratings.append(Rating(mu=player['overall']+player['elo']))
#             print(player['name'], ratings[-1])
        teams.append(ratings)
    print("{0} 胜率：".format(nations[0]), win_probability(teams[0], teams[1]))
    print("{0} 胜率：".format(nations[1]),win_probability(teams[1], teams[0]))
    print("平局概率：", quality(teams))
    print()

for item in matchups:
    calculate_with_matchup(item)

Argentina 胜率： 0.7880886535584259
Iceland 胜率： 0.2119113464415741
平局概率： 0.3462461430451184

Denmark 胜率： 0.5642746931879188
Peru 胜率： 0.4357253068120812
平局概率： 0.4425538370801158

